<p>
  <img src="https://upload.wikimedia.org/wikipedia/commons/f/fb/Escudo-UdeA.svg"
       alt="UdeA logo"
       height="190px"
       align="left"
       hspace="15px">
</p>

<h1 style="margin-top: 20px;"><b>6. Final project - Hall sensor, magnetic field detection - Arduino data</b></h1>

<hr>

<div align="right" style="font-size: 14px; line-height: 1.4;">
  <b>Estiven Castrillon Alzate</b><br>
  Institute of Physics<br>
  <i>Universidad de Antioquia</i><br>
  <span style="font-size: 12px;">Date: TO BE DEFINED</span>
</div>

<br><br>

# Abstract

This report details an experiment performed to measure the magnetic field of different magnets using a Hall effect sensor connected to an Arduino. The Hall effect is a physical phenomenon that occurs when an electric current flows through a conductor placed in a magnetic field. This experiment aims to analyze the relationship between distance and magnetic field, using a practical approach and the Arduino platform for data acquisition, specifically testing the decay of the magnetic field with distance experimentally.

Keywords: Hall effect, field, magnetism, Arduino, voltage, magnet.


# Introduction

The Hall effect, a physical phenomenon, has been a cornerstone in physics since its discovery in the late nineteenth century. First revealed by the American physicist Edwin Hall in 1879 [1], this effect has become fundamental in numerous technological applications and scientific experiments, playing a role in everyday technological instruments. The Hall effect occurs when an electric current flows through a conductor placed within a magnetic field that is perpendicular to the direction of the current, the consequence of this interaction is the generation of a transverse electric voltage, known as the Hall voltage, which appears perpendicular to both the magnetic field and the electric current. This seemingly simple phenomenon gives rise to a wide range of practical applications and highlights the deep connection between electricity and magnetism.

The discovery of the Hall effect by Edwin Hall not only expanded our understanding of the relationship between magnetic fields and electric currents, but also opened the door to numerous technological advances. Hall, an experimental physicist, observed this phenomenon while conducting experiments involving electric currents in the presence of magnetic fields. His observation of the lateral deflection of the electric current and the subsequent generation of a transverse potential difference marked the birth of the Hall effect.

This project utilizes the Arduino platform and an SS49E Hall effect sensor [2] to move from theory to practical experimentation, the primary objective beign analyzing the physical relationship between distance and magnetic field intensity by measuring the field decay of various magnets. By integrating Arduino with Python for data acquisition, the system converts analog signals into readable units, specifically Gauss (G) and milliTeslas (mT) based on the sensor's sensitivity of $1.4$ mV/Gauss as specified in [2]. Through experimental testing and statistical analysis using metrics such as $R^2$ and Mean Squared Error (MSE), this project seeks to validate the mathematical behavior of magnetic decay. Ultimately, the study aims to confirm whether the experimental data aligns with the theoretical inverse linear function (1/x), providing a deeper understanding of how magnetic fields propagate through space.

# Theoretical framework

The Hall effect is a physical phenomenon discovered by Edwin Hall in 1879. This effect appears when an electric current flows through a conductor in the presence of a magnetic field perpendicular to the current flow. As a result, a potential difference is generated, known as the Hall voltage, in the conductor along a direction perpendicular to both the magnetic field and the electric current.

### Fundamental principles of the Hall effect

**Lorentz force:** When an electric current flows through a conductor, the moving charges experience a force perpendicular to the magnetic field according to the Lorentz law.

**Generation of Hall voltage:** The Lorentz force causes a separation of charges within the conductor, creating a potential difference known as the Hall voltage.

**Direction of the Hall voltage:** The direction of the Hall voltage is perpendicular to both the direction of the current and the magnetic field, following the right hand rule.

### Applications of the Hall effect

**Magnetic field sensors:** The Hall effect is used in the manufacture of sensors to measure magnetic fields. Devices based on the Hall effect are highly sensitive and are used in a wide range of applications, from electronic compasses to magnetic switches.

**Motor control:** In the automotive industry and other fields, the Hall effect is used in position feedback systems to control the speed and position of motors.


## Experimental procedure

Initially, a system was developed using Arduino and Python code to control and obtain both digital and analog outputs from the SS49E Hall effect sensor (original data can be found in the folders `original_files`, `tests` and `sensor_hall_measurements_G-T`), as well as to transform the measurements provided by the sensor into units such as gauss (G) and millitesla (mT). The procedure was carried out as follows: a digital reading of the sensor was performed to determine whether it detected a magnetic field. If so, an LED was turned on to indicate the presence of the field. 

Next, an analog reading was taken to obtain a value proportional to the intensity of the magnetic field. However, this value was expressed on a scale determined by the resolution of the Arduino’s analog to digital converter. To convert this value into a more interpretable scale, it was transformed into voltage, considering the Arduino reference voltage of 5 V. 

After that, the magnetic field intensity was calculated in gauss using the sensor sensitivity, which has an average value of 1.4 mV/gauss according to the manufacturer (SS49E datasheet, 2019). This transformation allowed the measurement to be expressed in a commonly used magnetic field unit. Finally, the magnetic field intensity was converted from gauss to tesla by applying a conversion constant. This last conversion was essential to comply with the standards of the International System of Units (SI) and to provide a consistent measurement of the magnetic field in internationally recognized units, although in our analysis we used the unit mT for better data interpretation.

Subsequently, measurements were taken at different distances from the Hall effect sensor to the magnets labeled from 1 to 4. The first three magnets were radial ferrite magnets with radii of (0.075, 0.06, and 0.04 ± 0.001) meters, respectively, and the fourth magnet was rectangular.

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os
import warnings

from scipy.optimize import curve_fit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from plotly.subplots import make_subplots

warnings.simplefilter(action='ignore', category=FutureWarning)

Code to transform the data from .txt format to .csv format for processing with pandas. And also function to transform the data and normalize the database.

**Note: These can be used if you are reading the original data. I have uploaded the data to this notebook as json files inside the next cells, so you can skip this part if you want to work with the data directly.**


In [24]:
# # Path of the directory containing the text files
# path = '.'

# for txt_file in os.listdir(path):
#     if txt_file.endswith('.txt'):
#         # Get the file name without the extension
#         file_name = os.path.splitext(txt_file)[0]
        
#         # Read the file and save it as csv
#         file = pd.read_csv(os.path.join(path, txt_file), encoding='latin-1')
#         file.to_csv(os.path.join(path, f'{file_name}.csv'), index=None)

# def transform_columns(path: str):
#     for csv_file in os.listdir(path):
#         if csv_file.endswith('.csv'):
#             file_name = os.path.splitext(csv_file)[0]
#             data = pd.read_csv(
#                 os.path.join(path, csv_file),
#                 names=[
#                     'digital_state',
#                     'sensor_value',
#                     'voltage',
#                     'magnetic_field_gauss',
#                     'magnetic_field_millitesla'
#                 ]
#             )

#             # Iterate over columns and apply transformation to the values
#             for column in data:
#                 data[column] = data[column].apply(lambda x: x.split(':')[-1])

#             data.to_csv(os.path.join(path, f'{file_name}.csv'), index=None, sep=';')


# transform_columns(path)

In [25]:
# Data loading without using the original files
magnet_data_1 = pd.read_json('{"estado_digital":{"0":0,"1":0,"2":0,"3":0},"valor_sensor":{"0":533,"1":536,"2":551,"3":586},"voltaje":{"0":260.508,"1":261.975,"2":269.306,"3":286.413},"campo_gauss":{"0":0.00543,"1":0.01357,"2":0.0543,"3":0.14934},"campo_militeslas":{"0":0.00054,"1":0.00136,"2":0.00543,"3":0.01493},"distancia_metros":{"0":0.15,"1":0.1,"2":0.05,"3":0.01}}')

magnet_data_2 = pd.read_json('{"estado_digital":{"0":0,"1":0,"2":0,"3":0,"4":0},"valor_sensor":{"0":536,"1":539,"2":546,"3":559,"4":580},"voltaje":{"0":261.975,"1":263.441,"2":266.862,"3":273.216,"4":283.48},"campo_gauss":{"0":0.01629,"1":0.02444,"2":0.04345,"3":0.07875,"4":0.13577},"campo_militeslas":{"0":0.00163,"1":0.00244,"2":0.00434,"3":0.00787,"4":0.01358},"distancia_metros":{"0":0.1,"1":0.08,"2":0.06,"3":0.04,"4":0.02}}')

magnet_data_3 = pd.read_json('{"estado_digital":{"0":0,"1":0,"2":0,"3":0,"4":0},"valor_sensor":{"0":528,"1":527,"2":526,"3":525,"4":523},"voltaje":{"0":258.065,"1":257.576,"2":257.087,"3":256.598,"4":255.621},"campo_gauss":{"0":-0.00543,"1":-0.00814,"2":-0.01086,"3":-0.01357,"4":-0.019},"campo_militeslas":{"0":-0.00054,"1":-0.00081,"2":-0.00109,"3":-0.00136,"4":-0.0019},"distancia_metros":{"0":0.05,"1":0.04,"2":0.03,"3":0.02,"4":0.01}}')

magnet_data_4 = pd.read_json('{"estado_digital":{"0":0,"1":0,"2":0,"3":0},"valor_sensor":{"0":531,"1":533,"2":537,"3":547},"voltaje":{"0":259.531,"1":260.508,"2":262.463,"3":267.351},"campo_gauss":{"0":0.00272,"1":0.00815,"2":0.01901,"3":0.04616},"campo_militeslas":{"0":0.00027,"1":0.00081,"2":0.0019,"3":0.00462},"distancia_metros":{"0":0.05,"1":0.04,"2":0.03,"3":0.02}}')

# Renaming columns to english to not alter the original data
new_column_names = [
    'digital_state',
    'sensor_value',
    'voltage',
    'magnetic_field_gauss',
    'magnetic_field_millitesla',
    'distance_meters'
]

magnet_data_1.columns = new_column_names
magnet_data_2.columns = new_column_names
magnet_data_3.columns = new_column_names
magnet_data_4.columns = new_column_names

magnet_data_1.head()

# Preview of the data
display(magnet_data_1)

,digital_state,sensor_value,voltage,magnetic_field_gauss,magnetic_field_millitesla,distance_meters
0,0,533,260.508,0.00543,0.00054,0.15
1,0,536,261.975,0.01357,0.00136,0.10
2,0,551,269.306,0.05430,0.00543,0.05
3,0,586,286.413,0.14934,0.01493,0.01


In [26]:
# Data transformations and merging
total_data = pd.concat(
    [magnet_data_1, magnet_data_2, magnet_data_3, magnet_data_4],
    keys=['1', '2', '3', '4']
)

total_data = (
    total_data
    .reset_index(level=0)
    .rename(columns={'level_0': 'magnet'})
)

total_data['magnetic_field_millitesla'] = total_data['magnetic_field_millitesla'].apply(lambda x: abs(x))
total_data['magnetic_field_gauss'] = total_data['magnetic_field_gauss'].apply(lambda x: abs(x))
total_data['magnet'] = total_data['magnet'].astype(int)
total_data.reset_index(inplace=True, drop=True) # bad practice but made sure is okay to use in this context

total_data.head(20)

,magnet,digital_state,sensor_value,voltage,magnetic_field_gauss,magnetic_field_millitesla,distance_meters
0,1,0,533,260.508,0.00543,0.00054,0.15
1,1,0,536,261.975,0.01357,0.00136,0.10
2,1,0,551,269.306,0.05430,0.00543,0.05
3,1,0,586,286.413,0.14934,0.01493,0.01
4,2,0,536,261.975,0.01629,0.00163,0.10
5,2,0,539,263.441,0.02444,0.00244,0.08
6,2,0,546,266.862,0.04345,0.00434,0.06
7,2,0,559,273.216,0.07875,0.00787,0.04
8,2,0,580,283.480,0.13577,0.01358,0.02
9,3,0,528,258.065,0.00543,0.00054,0.05


# Results and analysis

In addition to the observation of a quadratic trend in the data collected using the Hall effect sensor, it is important to highlight that, according to our understanding of magnetic theory and the previous research conducted for this project, we identified that the magnetic field of the magnets exhibits a decay proportional to the inverse of the distance from the sensor to the magnets, that is, it follows a function of the form 1/x.

In [27]:
# Plot of magnetic fields as a function of distance
fig = px.line(
    total_data,
    x='distance_meters',
    y='magnetic_field_millitesla',
    color='magnet',
    labels={
        'distance_meters': 'Distance (meters)',
        'magnetic_field_millitesla': 'Magnetic field (millitesla)'
    },
    title='Magnetic field as a function of distance'
)

fig.update_layout(
    legend_title='Magnet',
    xaxis_title='Distance from sensor to magnet (m)',
    yaxis_title='Magnetic field (mT)',
    width=800,
    height=500,
)

fig.show()

Curve fitting with `curve_fit` of the data using the inverse function $\frac{1}{x}$ and evaluation metrics

In [28]:
total_data

,magnet,digital_state,sensor_value,voltage,magnetic_field_gauss,magnetic_field_millitesla,distance_meters
0,1,0,533,260.508,0.00543,0.00054,0.15
1,1,0,536,261.975,0.01357,0.00136,0.10
2,1,0,551,269.306,0.05430,0.00543,0.05
3,1,0,586,286.413,0.14934,0.01493,0.01
4,2,0,536,261.975,0.01629,0.00163,0.10
5,2,0,539,263.441,0.02444,0.00244,0.08
6,2,0,546,266.862,0.04345,0.00434,0.06
7,2,0,559,273.216,0.07875,0.00787,0.04
8,2,0,580,283.480,0.13577,0.01358,0.02
9,3,0,528,258.065,0.00543,0.00054,0.05


In [29]:
# Fitting functions
def func1(x, a, b):
    return a / x + b

def func2(x, a, b):
    return a / (x**2) + b

def func3(x, a, b):
    return a / (x**3) + b

# Fitting functions
functions = [func1, func2, func3]
function_names = ['Function 1/x', 'Function 1/x^2', 'Function 1/x^3']

rows = []  # collect rows here instead of using append

for magnet in [1, 2, 3, 4]:
    magnet_data = total_data[total_data['magnet'] == magnet]
    
    for function, name in zip(functions, function_names):
        popt, _ = curve_fit(function,
                            magnet_data['distance_meters'],
                            magnet_data['magnetic_field_millitesla'])
        
        y_pred = function(magnet_data['distance_meters'], *popt)

        r2 = r2_score(magnet_data['magnetic_field_millitesla'], y_pred)
        mse = mean_squared_error(magnet_data['magnetic_field_millitesla'], y_pred)
        mae = mean_absolute_error(magnet_data['magnetic_field_millitesla'], y_pred)

        rows.append({
            'Magnet': magnet,
            'Function': name,
            'R²': r2,
            'MSE': mse,
            'MAE': mae
        })

# Create the DataFrame once at the end
results_df = pd.DataFrame(rows)

results_df = results_df.sort_values(by='Function')
results_df

,Magnet,Function,R²,MSE,MAE
0,1,Function 1/x,0.959614,1.319139e-06,0.000967
3,2,Function 1/x,0.975689,4.642242e-07,0.000582
6,3,Function 1/x,0.917080,1.819931e-08,0.000117
9,4,Function 1/x,0.994224,1.623675e-08,0.000117
1,1,Function 1/x^2,0.913700,2.818869e-06,0.001368
4,2,Function 1/x^2,0.905349,1.807374e-06,0.001140
7,3,Function 1/x^2,0.817633,4.002596e-08,0.000173
10,4,Function 1/x^2,0.998581,3.987331e-09,0.000053
2,1,Function 1/x^3,0.899244,3.291059e-06,0.001466
5,2,Function 1/x^3,0.842458,3.008284e-06,0.001412


Statistical description of the behavior according to the metrics of the inverse function fits

In [30]:
results_df.groupby(by='Function').describe()

Magnet                                              R²  \
                count mean       std  min   25%  50%   75%  max count   
Function                                                                
Function 1/x      4.0  2.5  1.290994  1.0  1.75  2.5  3.25  4.0   4.0   
Function 1/x^2    4.0  2.5  1.290994  1.0  1.75  2.5  3.25  4.0   4.0   
Function 1/x^3    4.0  2.5  1.290994  1.0  1.75  2.5  3.25  4.0   4.0   

                          ...           MSE             MAE            \
                    mean  ...           75%       max count      mean   
Function                  ...                                           
Function 1/x    0.961652  ...  6.779530e-07  0.000001   4.0  0.000446   
Function 1/x^2  0.908816  ...  2.060248e-06  0.000003   4.0  0.000683   
Function 1/x^3  0.867486  ...  3.078978e-06  0.000003   4.0  0.000812   

                                                                            
                     std       min       25%       50%       75%       max  
Function                                                                    
Function 1/x    0.000411  0.000117  0.000117  0.000350  0.000679  0.000967  
Function 1/x^2  0.000667  0.000053  0.000143  0.000656  0.001197  0.001368  
Function 1/x^3  0.000724  0.000172  0.000193  0.000806  0.001425  0.001466  

[3 rows x 32 columns]

In [31]:
fig = make_subplots(
    rows=1,
    cols=3,
    subplot_titles=[
        'Fit of the form 1/x',
        'Fit of the form 1/x^2',
        'Fit of the form 1/x^3'
    ]
)

# Color palettes
colors_1 = px.colors.qualitative.Plotly
colors_2 = px.colors.qualitative.Dark2

# Definition of inverse functions
def func1(x, a, b):
    return a / (x) + b

def func2(x, a, b):
    return a / (x**2) + b

def func3(x, a, b):
    return a / (x**3) + b

# Iterate over subplots and magnets
for i, subplot_title, magnets, colors in zip([1, 2, 3], [''], [[1, 2, 3, 4]], [colors_1, colors_2]):
    for magnet, color in zip(magnets, colors):
        
        magnet_data = total_data[total_data['magnet'] == magnet]

        # Curve fitting
        popt, _ = curve_fit(func1, magnet_data['distance_meters'], magnet_data['magnetic_field_millitesla'])
        a_opt, b_opt = popt

        # Define fitted polynomial formula
        polynomial_formula = f"{a_opt:.6f}/x {b_opt:.6f}"

        # Generate data for fitted curve
        x_fit = np.linspace(
            magnet_data['distance_meters'].min(),
            magnet_data['distance_meters'].max(),
            100
        )
        y_fit = func1(x_fit, *popt)

        # Add scatter plot of real data
        scatter_trace = px.scatter(
            magnet_data,
            x='distance_meters',
            y='magnetic_field_millitesla',
            color='magnet'
        ).update_traces(marker=dict(color=color)).data[0]

        # Add fit line to subplot
        fit_trace = go.Scatter(
            x=x_fit,
            y=y_fit,
            mode='lines',
            name=f'Fit magnet {magnet}',
            line=dict(color=color, dash='dash')
        )
        fig.add_trace(scatter_trace, row=1, col=1)
        fig.add_trace(fit_trace, row=1, col=1)

for i, subplot_title, magnets, colors in zip([1, 2, 3], ['Magnets 1 and 2', 'Magnets 3 and 4'], [[1, 2, 3, 4]], [colors_1, colors_2]):
    for magnet, color in zip(magnets, colors):
        
        magnet_data = total_data[total_data['magnet'] == magnet]

        # Curve fitting
        popt, _ = curve_fit(func2, magnet_data['distance_meters'], magnet_data['magnetic_field_millitesla'])
        a_opt, b_opt = popt

        polynomial_formula = f"{a_opt:.6f}/x^2 {b_opt:.6f}"

        x_fit = np.linspace(
            magnet_data['distance_meters'].min(),
            magnet_data['distance_meters'].max(),
            100
        )
        y_fit = func2(x_fit, *popt)

        scatter_trace = px.scatter(
            magnet_data,
            x='distance_meters',
            y='magnetic_field_millitesla',
            color='magnet'
        ).update_traces(marker=dict(color=color)).data[0]

        fit_trace = go.Scatter(
            x=x_fit,
            y=y_fit,
            mode='lines',
            line=dict(color=color, dash='dash'),
            showlegend=False
        )
        fig.add_trace(scatter_trace, row=1, col=2)
        fig.add_trace(fit_trace, row=1, col=2)

for i, subplot_title, magnets, colors in zip([1, 2, 3], ['Magnets 1 and 2', 'Magnets 3 and 4'], [[1, 2, 3, 4]], [colors_1, colors_2]):
    for magnet, color in zip(magnets, colors):
        
        magnet_data = total_data[total_data['magnet'] == magnet]

        # Curve fitting
        popt, _ = curve_fit(func3, magnet_data['distance_meters'], magnet_data['magnetic_field_millitesla'])
        a_opt, b_opt = popt

        polynomial_formula = f"{a_opt:.6f}/x^3 {b_opt:.6f}"

        x_fit = np.linspace(
            magnet_data['distance_meters'].min(),
            magnet_data['distance_meters'].max(),
            100
        )
        y_fit = func3(x_fit, *popt)

        scatter_trace = px.scatter(
            magnet_data,
            x='distance_meters',
            y='magnetic_field_millitesla',
            color='magnet'
        ).update_traces(marker=dict(color=color)).data[0]

        fit_trace = go.Scatter(
            x=x_fit,
            y=y_fit,
            mode='lines',
            line=dict(color=color, dash='dash'),
            showlegend=False
        )
        fig.add_trace(scatter_trace, row=1, col=3)
        fig.add_trace(fit_trace, row=1, col=3)

fig.update_layout(
    title_text='Fits of the model to the sensor distance data versus the measured magnetic field in millitesla (mT)',
    legend_title='Fits',
    xaxis_title='Distance from the sensor to the magnet (m)',
    yaxis_title='Magnetic field (mT)',
    xaxis2_title='Distance from the sensor to the magnet (m)',
    xaxis3_title='Distance from the sensor to the magnet (m)',
    width=1200,
    height=500,
)

# fig.write_image("fits_x_inverse.png")
fig.show()


**Figure 1:** Fits with inverse functions of the experimental data  

As shown in Figure 1, the three functions appear to fit the experimental data well. However, we proceeded with a statistical analysis using three metrics to determine which was the best possible fit for our data. These metrics were R², Mean Squared Error (MSE), and Mean Absolute Error (MAE).

On average, for the four magnets, the metrics for the 1/x function were clearly superior, with the following values: R² = 0.96, MSE = 4.5 × 10⁻⁷, and MAE = 4.46 × 10⁻⁴.

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=['Magnets 1 and 2', 'Magnets 3 and 4'])

# Color palettes
colors_1 = px.colors.qualitative.Plotly
colors_2 = px.colors.qualitative.Set2

# Polynomial fitting function
def polynomial_fit(x, y, degree, color):
    coefficients = np.polyfit(x, y, degree)
    polynomial = np.poly1d(coefficients)
    formula = str(polynomial).replace('x', 'x^2', 1).split('\n')[-1]
    return polynomial, color, formula

for i, subplot_title, magnets, colors in zip(
    [1, 2],
    ['Magnet 1 and 2', 'Magnet 3 and 4'],
    [[1, 2], [3, 4]],
    [colors_1, colors_2]
):
    for magnet, color in zip(magnets, colors):
        magnet_data = total_data[total_data['magnet'] == magnet]
        
        # Second degree polynomial fit
        polynomial, curve_color, formula = polynomial_fit(
            magnet_data['distance_meters'],
            magnet_data['magnetic_field_millitesla'],
            degree=2,
            color=color
        )
        x_vals = np.linspace(
            min(magnet_data['distance_meters']),
            max(magnet_data['distance_meters']),
            100
        )
        y_vals = polynomial(x_vals)
        
        # Add fitted curve as an additional line
        fig.add_trace(
            px.line(x=x_vals, y=y_vals, line_shape='linear')
            .update_traces(line=dict(color=curve_color))
            .data[0],
            row=1, col=i
        )
        
        # Add original data as points
        fig.add_trace(
            go.Scatter(
                x=magnet_data['distance_meters'],
                y=magnet_data['magnetic_field_millitesla'],
                mode='markers',
                marker=dict(color=color),
                name=f'Original data magnet {magnet}'
            ),
            row=1, col=i
        )

fig.update_layout(
    title_text='Second degree polynomial fits for the distance data in relation to the magnetic field',
    legend_title='Polynomial fits',
    xaxis_title='Distance from the sensor to the magnet (m)',
    yaxis_title='Magnetic field (mT)',
    xaxis2_title='Distance from the sensor to the magnet (m)',
    width=1200,
    height=500
)

# fig.write_image("polynomial_fits.png")

fig.show()


**Figure 2:** Distribution of the experimental data  

With the aim of experimentally validating this physical phenomenon and the experimental trend shown in Figure 1, we carried out a deeper analysis using Python’s `curve_fit` function. This function allows us to fit our data to different powers of the function 1/x, thus exploring the optimal relationship between distance and magnetic field intensity. By implementing three fits with different powers, we were able to determine more precisely the behavior of the magnetic field as a function of distance.

# Conclusions

The results obtained from the experiment with the Hall effect sensor provide valuable insight into the variation of the magnetic field of the magnets as a function of the distance to the sensor. The observations and analyses carried out allow the following conclusions to be drawn:

First, the identification of a quadratic trend in the data supports the theoretical expectations based on magnetic theory and prior research. In addition, the experimental analysis performed using Python’s `curve_fit` function facilitated fitting the data to three different powers of the inverse function. This approach provided a detailed understanding of the relationship between distance and magnetic field intensity.

Despite the limitation in the amount of data collected, the linear inverse function 1/x emerged as the best average fit for the four magnets. The consistency of the metrics across the different magnets reinforces the validity and robustness of the methodology used. Furthermore, it highlights the general applicability of the 1/x function to describe the decay of the magnetic field in this experimental context.

In conclusion, the findings strongly support the selection of the linear inverse function 1/x as an accurate model for describing the decay of the magnetic field in the experimental setup.

# References

[1] E. H. Hall, “On a new action of the magnet on electric currents,” American Journal of Science, vol. s3-19, no. 111, pp. 200–205, Mar. 1880, doi: https://doi.org/10.2475/ajs.s3-19.111.200.

[2] “SS49E Datasheet - Linear Hall Effect Sensor | SEC Electronics Datasheet,” Datasheet4u.com, 2026. https://datasheet4u.com/datasheets/SEC-Electronics/SS49E/724330 (accessed Feb. 15, 2026).
‌